In [259]:
import pandas as pd
import json
import csv

USERS DATA PREPROCESSING

In [260]:
users = pd.read_json('users.json',lines=True)

In [261]:

users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


In [262]:
def extract_value(row, key):
    value = row.get(key) if isinstance(row, dict) else None
    if value and key == '$date':
        return str(value)
    return value

In [263]:

# Extracting '_id', 'createdDate', 'lastLogin' columns
users['_id'] = users['_id'].apply(lambda x: extract_value(x, '$oid'))
users['createdDate'] = users['createdDate'].apply(lambda x: extract_value(x, '$date'))
users['lastLogin'] = users['lastLogin'].apply(lambda x: extract_value(x, '$date'))

In [264]:

# Converting 'createdDate' and 'lastLogin' to datetime
users['createdDate'] = pd.to_datetime(users['createdDate'], unit='ms')
users['lastLogin'] = pd.to_datetime(users['lastLogin'], unit='ms')
date_columns = ['createdDate', 'lastLogin']
for column in date_columns:
  users[column] = pd.to_datetime(users[column], errors='coerce').where(pd.notnull(users[column]), None)

In [265]:

users
     

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204,fetch-staff,NaN,NaN


In [266]:

users = users.rename(columns={'_id': 'userId'})

In [267]:

users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   userId        495 non-null    object        
 1   active        495 non-null    bool          
 2   createdDate   495 non-null    datetime64[ns]
 3   lastLogin     433 non-null    datetime64[ns]
 4   role          495 non-null    object        
 5   signUpSource  447 non-null    object        
 6   state         439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB


In [268]:
# Removing duplicate records
initial_record_count = len(users)
users.drop_duplicates(inplace=True)
final_record_count = len(users)
print(f"Number of records before removing duplicates: {initial_record_count}")
print(f"Number of records after removing duplicates: {final_record_count}")

# Checking for null or duplicate values in userId
user_id_null_count = users['userId'].isnull().sum()
user_id_duplicate_count = users['userId'].duplicated().sum()
print(f"Number of null values in userId: {user_id_null_count}")
print(f"Number of duplicate values in userId: {user_id_duplicate_count}")

Number of records before removing duplicates: 495
Number of records after removing duplicates: 212
Number of null values in userId: 0
Number of duplicate values in userId: 0


In [269]:


#Checking for any invalid data in createdDate and lastLogin columns
invalid_date_records = users[users['createdDate'] > users['lastLogin']]
print(f"Records where createdDate is later than lastLogin date:\n{invalid_date_records}")

#Checking the unique values in the column 'role'
unique_role_values = users['role'].unique()
print(f"Unique values in 'role' column: {unique_role_values}")

#Checking the unique values in the column 'signupSource'
unique_sign_up_sources = users['signUpSource'].unique()
print(f"Unique values in 'signUpSource' column: {unique_sign_up_sources}")

# Ensuring 'active' column contains only boolean values
unique_active_values = users['active'].unique()
print(f"Unique values in 'active' column: {unique_active_values}")

#Checking null values in columns createdDate and lastLogin columns
createdDate_null_values = users[users['createdDate'].isnull()][['userId', 'createdDate']]
lastLogin_null_values = users[users['lastLogin'].isnull()][['userId','lastLogin']]
print(f"Records with null values in createdDate column:\n{createdDate_null_values}")
print(f"Records with null values in lastLogin column:\n{lastLogin_null_values}")


Records where createdDate is later than lastLogin date:
Empty DataFrame
Columns: [userId, active, createdDate, lastLogin, role, signUpSource, state]
Index: []
Unique values in 'role' column: ['consumer' 'fetch-staff']
Unique values in 'signUpSource' column: ['Email' 'Google' nan]
Unique values in 'active' column: [ True False]
Records with null values in createdDate column:
Empty DataFrame
Columns: [userId, createdDate]
Index: []
Records with null values in lastLogin column:
                       userId lastLogin
97   5ff616a68f142f11dd189163       NaT
143  5ffe115404929101d0aaebb2       NaT
170  5e27526d0bdb6a138c32b556       NaT
180  6002475cfb296c121a81b98d       NaT
181  60024f24e257124ec6b99a13       NaT
182  60037329e257124ec6b99eeb       NaT
183  6000b75bbe5fc96dfee1d4d3       NaT
184  6002541ae257124ec6b99a3a       NaT
186  600258dafb296c4ef805d9b7       NaT
187  60025c65fb296c4ef805d9e6       NaT
188  60037005e257124ec6b99ec2       NaT
195  60023de5fb296c121a81b955       NaT


In [270]:

# Validating 'state' column
valid_state_codes = set(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'])
invalid_states = users[~users['state'].isin(valid_state_codes) | users['state'].isnull()][['userId', 'state']]
print(f"Records with null values in the 'state' column:\n{invalid_states}")

Records with null values in the 'state' column:
                       userId state
344  60145ff384231211ce796d51   NaN
375  60186237c8b50e11d8454d5f   NaN
422  5a43c08fe4b014fd6b6a0612   NaN
432  5fbc35711d967d1222cbfefc   NaN
455  5fa41775898c7a11a6bcef3e   NaN
475  54943462e4b07e684157a532   NaN


In [271]:

null_values_count = users.isnull().sum()
null_values_count

userId           0
active           0
createdDate      0
lastLogin       40
role             0
signUpSource     5
state            6
dtype: int64

In [272]:
# Calculate percentages for state column
state_percentages = (users['state'].value_counts(normalize=True) * 100).round(2)



# Display results
print("State Distribution (%):")
print(state_percentages)


State Distribution (%):
WI    93.69
AL     2.43
IL     1.46
KY     0.49
CO     0.49
OH     0.49
SC     0.49
NH     0.49
Name: state, dtype: float64


In [273]:
# Calculate percentages for signUpSource column
signup_percentages = (users['signUpSource'].value_counts(normalize=True) * 100).round(2)
print("\nSign Up Source Distribution (%):")
print(signup_percentages)


Sign Up Source Distribution (%):
Email     98.55
Google     1.45
Name: signUpSource, dtype: float64



Data PreProcessing and Exploratory data analysis report:


1. Duplicate records: Over 50% redundancy in Users data.
2.Missing values found in columns signUpSource, State and lastLogin.
3. Majority of signups are through email and most of the users are resided in WI.


BRANDS DATA PREPROCESSING

In [274]:
brands = pd.read_json('brands.json',lines=True)


In [275]:

brands

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


In [276]:
def extract_info(row):
    brandId = extract_value(row['_id'], '$oid')
    cpgId = extract_value(row['cpg'].get('$id'), '$oid') if isinstance(row['cpg'], dict) else None
    cpgRef = extract_value(row['cpg'], '$ref') if isinstance(row['cpg'], dict) else None
    return brandId, cpgId, cpgRef

In [277]:

brands[['brandId', 'cpgId', 'cpgRef']] = brands.apply(lambda row: pd.Series(extract_info(row)), axis=1)

# Dropping the original '_id' and 'cpg' columns
brands = brands.drop(columns=['_id', 'cpg'])

In [278]:

brands

,barcode,category,categoryCode,name,topBrand,brandCode,brandId,cpgId,cpgRef
0,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs


In [279]:

brands.info()
     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   barcode       1167 non-null   int64  
 1   category      1012 non-null   object 
 2   categoryCode  517 non-null    object 
 3   name          1167 non-null   object 
 4   topBrand      555 non-null    float64
 5   brandCode     933 non-null    object 
 6   brandId       1167 non-null   object 
 7   cpgId         1167 non-null   object 
 8   cpgRef        1167 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 82.2+ KB


In [280]:
# Converting values in 'topBrand' column to boolean
brands['topBrand'] = brands['topBrand'].replace({1: True, 0: False})

# Removing duplicate records
initial_record_count = len(brands)
brands.drop_duplicates(inplace=True)
final_record_count = len(brands)
print(f"Number of records before removing duplicates: {initial_record_count}")
print(f"Number of records after removing duplicates: {final_record_count}")



Number of records before removing duplicates: 1167
Number of records after removing duplicates: 1167


In [281]:

# Checking for null or duplicate values in brandId
brand_id_null_count = brands['brandId'].isnull().sum()
brand_id_duplicate_count = brands['brandId'].duplicated().sum()
print(f"Number of null values in brandId: {brand_id_null_count}")
print(f"Number of duplicate values in brandId: {brand_id_duplicate_count}")

# Ensuring 'topBrand' column contains only boolean values
unique_topBrand_values = brands['topBrand'].unique()
print(f"Unique values in 'topBrand' column: {unique_topBrand_values}")
brands['topBrand'] = brands['topBrand'].astype('boolean')
brands['topBrand'] = brands['topBrand'].where(pd.notnull(brands['topBrand']), None)


Number of null values in brandId: 0
Number of duplicate values in brandId: 0
Unique values in 'topBrand' column: [False nan True]


In [282]:

null_values_count = brands.isnull().sum()
null_values_count
     

barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
brandId           0
cpgId             0
cpgRef            0
dtype: int64

In [283]:
# Find the number of unique values for the specified columns
unique_categoryCode_values = brands['categoryCode'].nunique()
unique_category_values = brands['category'].nunique()
unique_barcode_values = brands['barcode'].nunique()
unique_brandCode_values = brands['brandCode'].nunique()

# Display the number of unique values for each column
print("Number of unique values in 'categoryCode' column:", unique_categoryCode_values)
print("Number of unique values in 'category' column:", unique_category_values)
print("Number of unique values in 'barcode' column:", unique_barcode_values)
print("Number of unique values in 'brandCode' column:", unique_brandCode_values)


duplicate_barcodes = brands[brands.duplicated(subset=['barcode'], keep=False)]

# Find records where brandCode column has nulls or zeros
null_or_zero_brandcode = brands[(brands['brandCode'].isnull()) | (brands['brandCode'] == 0)]

# Get the count of each occurrence
duplicate_barcodes_count = duplicate_barcodes.shape[0]
null_or_zero_brandcode_count = null_or_zero_brandcode.shape[0]
# Display the number of unique values for each column
print("Number of duplicate barcode values:", duplicate_barcodes_count)
print("Number of duplicate brandcode values:", null_or_zero_brandcode_count)
null_or_zero_brandcode

Number of unique values in 'categoryCode' column: 14
Number of unique values in 'category' column: 23
Number of unique values in 'barcode' column: 1160
Number of unique values in 'brandCode' column: 897
Number of duplicate barcode values: 14
Number of duplicate brandcode values: 234


,barcode,category,categoryCode,name,topBrand,brandCode,brandId,cpgId,cpgRef
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
11,511111102540,NaN,NaN,MorningStar,<NA>,NaN,57c08106e4b0718ff5fcb02c,5332f5f2e4b03c9a25efd0aa,Cpgs
18,511111317364,Baking,BAKING,test brand @1605535049181,False,NaN,5fb28549be37ce522e165cb5,5fb28549be37ce522e165cb4,Cogs
23,511111303947,NaN,NaN,Bottled Starbucks,<NA>,NaN,5332f5fee4b03c9a25efd0bd,53e10d6368abd3c7065097cc,Cpgs
24,511111802914,NaN,NaN,Full Throttle,<NA>,NaN,5332fa7ce4b03c9a25efd22e,5332f5ebe4b03c9a25efd0a8,Cpgs
...,...,...,...,...,...,...,...,...,...
1135,511111405184,NaN,NaN,Do It Yourself,<NA>,NaN,5d658fca6d5f3b23d1bc7912,53e10d6368abd3c7065097cc,Cogs
1144,511111202516,NaN,NaN,Corona,<NA>,NaN,57c08242e4b0718ff5fcb032,5332f7a7e4b03c9a25efd134,Cpgs
1146,511111703105,NaN,NaN,Bellatoria,<NA>,NaN,5332fa12e4b03c9a25efd1e6,5332fa12e4b03c9a25efd1e7,Cpgs
1157,511111303015,NaN,NaN,DASANI,<NA>,NaN,5332fa75e4b03c9a25efd221,5332f5ebe4b03c9a25efd0a8,Cpgs


In [284]:

brands
     

,barcode,category,categoryCode,name,topBrand,brandCode,brandId,cpgId,cpgRef
0,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,False,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,<NA>,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,<NA>,DIPPIN DOTS CEREAL,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,<NA>,TEST BRANDCODE @1598639215217,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,LIPTON TEA Leaves,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs


checking if each brandCode has an unique brandId

In [285]:
# Group by brandCode and get unique brandId values for each
brand_mapping = brands.groupby('brandCode')['brandId'].unique()

# Display the results
print("Unique brandId values for each brandCode:")
print(brand_mapping)

# To see how many unique brandIds each brandCode has
brand_counts = brands.groupby('brandCode')['brandId'].nunique()
print("\nNumber of unique brandIds per brandCode:")
print(brand_counts)

Unique brandId values for each brandCode:
brandCode
                      [57ebc11fe4b0ac389136a33a, 58861c7d4e8d0d20bc4...
 MILANO                                      [5ab13217e4b0915382df761b]
 SOBE                                        [5d66e20b6d5f3b6188d4f055]
 TAZO BOTTLED TEAS                           [5d66e2d46d5f3b6188d4f056]
09090909090                                  [5c4637ba87ff35681e840d57]
                                            ...                        
YOPLAIT GO-GURT                              [5daf6baaa60b87376833e364]
YQ YOPLAIT                                   [5daf43251dda2c3e1416ae95]
YUBAN                                        [580dffade4b0f32b2de21384]
ZIMA                                         [5952a596e4b08f39def64d03]
ZUMBIDA                                      [5a5d28f1e4b0db471c2d0432]
Name: brandId, Length: 897, dtype: object

Number of unique brandIds per brandCode:
brandCode
                      35
 MILANO                1
 SOBE       

In [286]:
# Filter out nulls and empty/whitespace strings in brandCode, and nulls in brandId
clean_brands = brands[
    (brands['brandCode'].notna()) & 
    (brands['brandCode'].str.strip() != '') & 
    (brands['brandId'].notna())
]

# Group by brandCode and get unique brandId values for each
brand_mapping = clean_brands.groupby('brandCode')['brandId'].unique()

# Display the results
print("Unique brandId values for each brandCode:")
print(brand_mapping)

# To see how many unique brandIds each brandCode has
brand_counts = clean_brands.groupby('brandCode')['brandId'].nunique()
print("\nNumber of unique brandIds per brandCode:")
print(brand_counts)

Unique brandId values for each brandCode:
brandCode
 MILANO               [5ab13217e4b0915382df761b]
 SOBE                 [5d66e20b6d5f3b6188d4f055]
 TAZO BOTTLED TEAS    [5d66e2d46d5f3b6188d4f056]
09090909090           [5c4637ba87ff35681e840d57]
0987654321            [5c45f91b87ff3552f950f027]
                                 ...            
YOPLAIT GO-GURT       [5daf6baaa60b87376833e364]
YQ YOPLAIT            [5daf43251dda2c3e1416ae95]
YUBAN                 [580dffade4b0f32b2de21384]
ZIMA                  [5952a596e4b08f39def64d03]
ZUMBIDA               [5a5d28f1e4b0db471c2d0432]
Name: brandId, Length: 896, dtype: object

Number of unique brandIds per brandCode:
brandCode
 MILANO               1
 SOBE                 1
 TAZO BOTTLED TEAS    1
09090909090           1
0987654321            1
                     ..
YOPLAIT GO-GURT       1
YQ YOPLAIT            1
YUBAN                 1
ZIMA                  1
ZUMBIDA               1
Name: brandId, Length: 896, dtype: int64



Data PreProcessing and Exploratory data analysis report:


1. Duplicate records: No duplicate records are found.
2. Conisderable missing value in category, topBrand, brandCode columns.
category        155
categoryCode    650
topBrand        612
brandCode       234
3. Each brandCode has an unique brandId but few brandCodes are missing for some brandId's

RECEIPTS DATA PROCESSING

In [287]:
receipts = pd.read_json('receipts.json',lines=True)

In [288]:

# Extracting '_id', 'createdDate', 'lastLogin' columns
nested_columns=['_id','createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate']
for column in nested_columns:
  if column=='_id':
    key='$oid'
  else:
    key='$date'
  receipts[column]=receipts[column].apply(extract_value,key=key)

receipts = receipts.rename(columns = {"_id":"receiptId"})


date_columns = ['createDate', 'dateScanned', 'finishedDate','modifyDate', 'pointsAwardedDate', 'purchaseDate']
for column in date_columns:
    receipts[column] = pd.to_datetime(receipts[column], unit='ms')
    receipts[column] = pd.to_datetime(receipts[column], errors='coerce').where(pd.notnull(receipts[column]), None)

In [289]:
receipts

,receiptId,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:36.000,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaT,2021-01-03 15:25:42.000,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:39.000,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:11.000,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NaT,2021-03-01 10:22:28.000,NaT,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [290]:



rewards_items=[]
for idx, row in receipts.iterrows():
    receipt_id = row["receiptId"]
    items = row["rewardsReceiptItemList"]

    # Ensure items is a list before iterating
    if isinstance(items, list):
        for item in items:
            item["receiptId"] = receipt_id
            rewards_items.append(item)

items=pd.DataFrame(rewards_items)
receipts = receipts.drop(columns=['rewardsReceiptItemList',])

In [291]:

receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   receiptId                1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    float64       
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptStatus     1119 non-null   object        
 12  totalSpent               684 non-n

In [292]:
# Removing duplicate records
initial_record_count = len(receipts)
receipts.drop_duplicates(inplace=True)
final_record_count = len(receipts)
print(f"Number of records before removing duplicates: {initial_record_count}")
print(f"Number of records after removing duplicates: {final_record_count}")

# Checking for null or duplicate values in receiptId
receipt_id_null_count = receipts['receiptId'].isnull().sum()
receipt_id_duplicate_count = receipts['receiptId'].duplicated().sum()
print(f"Number of null values in receiptId: {receipt_id_null_count}")
print(f"Number of duplicate values in receiptId: {receipt_id_duplicate_count}")

#Checking the unique values in the column 'rewardsReceiptStatus'
unique_status_values = receipts['rewardsReceiptStatus'].unique()
print(f"Unique values in 'rewards receipt status' column: {unique_status_values}")


# Find user IDs from receipts that are not present in the user table
missing_user_ids = receipts[~receipts['userId'].isin(users['userId'])]
missing_user_count = len(missing_user_ids)

# Display the missing user IDs and count
print("Missing User IDs:")
print(missing_user_ids[['userId']])
print("\nNumber of Missing User IDs:", missing_user_count)

#Dropping the userIds which are not present in users table
receipts = receipts.drop(missing_user_ids.index)

Number of records before removing duplicates: 1119
Number of records after removing duplicates: 1119
Number of null values in receiptId: 0
Number of duplicate values in receiptId: 0
Unique values in 'rewards receipt status' column: ['FINISHED' 'REJECTED' 'FLAGGED' 'SUBMITTED' 'PENDING']
Missing User IDs:
                       userId
13   5f9c74f7c88c1415cbddb839
15   5ff1e1e9b6a9d73a3a9f10f6
16   5ff1e1dfcfcf6c399c274ab3
20   5f9c74e3f1937815bd2c1d73
21   5ff1e196cfcf6c399c274a38
..                        ...
955  60253861efa6017a44dc6b50
956  60253891b54593795bf69242
966  60253891b54593795bf69242
985  60268c7bb545931ac63683af
990  60268c78efa6011bb151077d

[148 rows x 1 columns]

Number of Missing User IDs: 148


In [293]:

#Checking for any invalid data in createdDate and lastLogin columns
invalid_date_records = receipts[receipts['purchaseDate'] > receipts['pointsAwardedDate']]
print(f"Records where purchaseDate is later than pointsAwardedDate date:\n{invalid_date_records}")


Records where purchaseDate is later than pointsAwardedDate date:
                    receiptId  bonusPointsEarned  \
14   5ff1e1b20a7214ada100055a              300.0   
85   5ff4ce640a7214ada10005e0               25.0   
362  600887560a720f05fa000098              250.0   
553  60145a3d0a7214ad50000082              750.0   

                               bonusPointsEarnedReason          createDate  \
14   Receipt number 4 completed, bonus point schedu... 2021-01-03 15:24:34   
85                         COMPLETE_NONPARTNER_RECEIPT 2021-01-05 20:39:00   
362  Receipt number 3 completed, bonus point schedu... 2021-01-20 19:41:10   
553  Receipt number 1 completed, bonus point schedu... 2021-01-29 18:55:57   

            dateScanned        finishedDate          modifyDate  \
14  2021-01-03 15:24:34 2021-01-03 15:24:35 2021-01-03 15:24:35   
85  2021-01-05 20:39:00 2021-01-05 20:39:00 2021-01-05 20:39:00   
362 2021-01-20 19:41:10 2021-01-20 19:41:11 2021-01-20 19:41:11   
553 2021-01-29 

In [294]:

null_values_count = receipts.isnull().sum()
null_values_count
     

receiptId                    0
bonusPointsEarned          553
bonusPointsEarnedReason    553
createDate                   0
dateScanned                  0
finishedDate               512
modifyDate                   0
pointsAwardedDate          552
pointsEarned               495
purchaseDate               434
purchasedItemCount         480
rewardsReceiptStatus         0
totalSpent                 431
userId                       0
dtype: int64

In [295]:

receipts

,receiptId,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:36.000,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaT,2021-01-03 15:25:42.000,NaT,5.0,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:39.000,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:11.000,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NaT,2021-03-01 10:22:28.000,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [296]:

receipts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 971 entries, 0 to 1118
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   receiptId                971 non-null    object        
 1   bonusPointsEarned        418 non-null    float64       
 2   bonusPointsEarnedReason  418 non-null    object        
 3   createDate               971 non-null    datetime64[ns]
 4   dateScanned              971 non-null    datetime64[ns]
 5   finishedDate             459 non-null    datetime64[ns]
 6   modifyDate               971 non-null    datetime64[ns]
 7   pointsAwardedDate        419 non-null    datetime64[ns]
 8   pointsEarned             476 non-null    float64       
 9   purchaseDate             537 non-null    datetime64[ns]
 10  purchasedItemCount       491 non-null    float64       
 11  rewardsReceiptStatus     971 non-null    object        
 12  totalSpent               540 non-nu


Data PreProcessing and Exploratory data analysis report:


1. duplicate records: No duplicate records are found.
2. conisderable missing values in multiple columns.
3. rewardsReceiptItemList' field as it contains lists of receipts, so moving all the data here into new dataframe items.
4. date columns the time is stored as UTC so we convert that into datetime format 
5. missing values in  userId column.


ITEM DATA PROCESSING

In [297]:

items
     

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [298]:

#Dropping rceeiptIds which are not present in receipt table
missing_receipts = items[~items['receiptId'].isin(receipts['receiptId'])]
items = items.drop(missing_receipts.index)

In [299]:

items['barcode'] = pd.to_numeric(items['barcode'], errors='coerce').fillna(0).astype(int)

In [300]:

items.isnull().sum()

barcode                                  0
description                            212
finalPrice                              55
itemPrice                               55
needsFetchReview                      4182
partnerItemId                            0
preventTargetGapPoints                4172
quantityPurchased                       55
userFlaggedBarcode                    4193
userFlaggedNewItem                    4202
userFlaggedPrice                      4222
userFlaggedQuantity                   4222
receiptId                                0
needsFetchReviewReason                4256
pointsNotAwardedReason                4117
pointsPayerId                         3417
rewardsGroup                          3055
rewardsProductPartnerId               2681
userFlaggedDescription                4264
originalMetaBriteBarcode              4282
originalMetaBriteDescription          4336
brandCode                             2523
competitorRewardsGroup                4170
discountedI

In [301]:
items.info()
     

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4337 entries, 0 to 6940
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   barcode                             4337 non-null   int32  
 1   description                         4125 non-null   object 
 2   finalPrice                          4282 non-null   object 
 3   itemPrice                           4282 non-null   object 
 4   needsFetchReview                    155 non-null    object 
 5   partnerItemId                       4337 non-null   object 
 6   preventTargetGapPoints              165 non-null    object 
 7   quantityPurchased                   4282 non-null   float64
 8   userFlaggedBarcode                  144 non-null    object 
 9   userFlaggedNewItem                  135 non-null    object 
 10  userFlaggedPrice                    115 non-null    object 
 11  userFlaggedQuantity                 115 non

In [302]:
initial_record_count = len(items)
receipts.drop_duplicates(inplace=True)
final_record_count = len(items)
print(f"Number of records before removing duplicates: {initial_record_count}")
print(f"Number of records after removing duplicates: {final_record_count}")

Number of records before removing duplicates: 4337
Number of records after removing duplicates: 4337


In [303]:

items['competitiveProduct'] = items['competitiveProduct'].astype('boolean')
items['competitiveProduct'] = items['competitiveProduct'].where(pd.notnull(items['competitiveProduct']), None)
items['deleted'] = items['deleted'].astype('boolean')
items['deleted'] = items['deleted'].where(pd.notnull(items['deleted']), None)
items['needsFetchReview'] = items['needsFetchReview'].astype('boolean')
items['needsFetchReview'] = items['needsFetchReview'].where(pd.notnull(items['needsFetchReview']), None)
items['preventTargetGapPoints'] = items['preventTargetGapPoints'].astype('boolean')
items['preventTargetGapPoints'] = items['preventTargetGapPoints'].where(pd.notnull(items['preventTargetGapPoints']), None)
items['userFlaggedNewItem'] = items['userFlaggedNewItem'].astype('boolean')
items['userFlaggedNewItem'] = items['userFlaggedNewItem'].where(pd.notnull(items['userFlaggedNewItem']), None)



Data PreProcessing and Exploratory data analysis report:


1. duplicate records: No duplicate records are found.
2. conisderable missing values in multiple columns except for barcode, partnerItemId, receiptId  .


Exporting all the preprocessed dataframes into csv files for data ingestion into a postgresql database.

In [304]:

users.to_csv("users.csv",index=False)

In [305]:

brands.to_csv("brands.csv",index=False)

In [306]:

receipts.to_csv("receipts.csv",index=False)

In [307]:

items.to_csv("items.csv",index=False)
     
    

In [308]:
items.to_csv("C:/Users/upend/Desktop/FetchRewards_TakeHome_Project/Data/items.csv", index=False)



In [309]:
users.to_csv("C:/Users/upend/Desktop/FetchRewards_TakeHome_Project/Data/users.csv", index=False)

In [310]:
brands.to_csv("C:/Users/upend/Desktop/FetchRewards_TakeHome_Project/Data/brands.csv", index=False)

In [311]:
receipts.to_csv("C:/Users/upend/Desktop/FetchRewards_TakeHome_Project/Data/receipts.csv", index=False)